# 📊 Book de Variáveis: Dados Faturamento

**Objetivo:** Gerar ~120 variáveis de faturamento para modelos de ML (Risco, Churn, Propensão).  
**Granularidade:** `NUM_CPF` + `SAFRA`  
**Tabela Fato:** `rawdata.dados_faturamento`  
**Dimensões:** `plataforma`, `tipo_faturamento`, `dados_cadastrais`  
**Data de Geração:** 2026-01-15

In [ ]:
# =============================================================================
# PARAMETROS DO NOTEBOOK (SAFRAS)
# =============================================================================
from datetime import date

# Lista de safras a processar
LISTA_SAFRAS = ['202410', '202411', '202412', '202501', '202502', '202503']
MODO_ESCRITA = 'overwrite'

def safra_to_cutoff(safra):
    '''Converte safra (yyyyMM) para data de cutoff. Ex: 202410 -> 2024-10-01'''
    return date(int(safra[:4]), int(safra[4:6]), 1).strftime('%Y-%m-%d')

print('Configuracao de Safras:')
for s in LISTA_SAFRAS:
    print(f'  Safra {s}: dados < {safra_to_cutoff(s)}')

## 1. Setup e Configurações

In [ ]:
# =============================================================================
# CONFIGURAÇÃO DO AMBIENTE (importado do config centralizado)
# =============================================================================
import sys; sys.path.insert(0, "/lakehouse/default/Files/projeto-final")
from config.pipeline_config import SILVER_BASE, PATH_BOOK_FATURAMENTO

# Configurações do Spark
spark.conf.set("spark.sql.shuffle.partitions", "200")
spark.conf.set("spark.sql.adaptive.enabled", "true")

LAKEHOUSE_PATH = SILVER_BASE

# Paths de Entrada
PATH_FATURAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/dados_faturamento"
PATH_PLATAFORMA = f"{LAKEHOUSE_PATH}/Tables/rawdata/plataforma"
PATH_TIPO_FATURAMENTO = f"{LAKEHOUSE_PATH}/Tables/rawdata/tipo_faturamento"

# Path de Saída
PATH_OUTPUT = PATH_BOOK_FATURAMENTO

print("✅ Configuração carregada")
print(f"📁 Output: {PATH_OUTPUT}")

## 2. Carregamento das Tabelas

In [ ]:
# =============================================================================
# CARREGAMENTO DAS TABELAS DELTA
# =============================================================================

df_faturamento = spark.read.format("delta").load(PATH_FATURAMENTO)
print(f"📊 faturamento: {df_faturamento.count():,} registros")

df_plataforma = spark.read.format("delta").load(PATH_PLATAFORMA)
print(f"📊 plataforma: {df_plataforma.count():,} registros")

df_tipo_faturamento = spark.read.format("delta").load(PATH_TIPO_FATURAMENTO)
print(f"📊 tipo_faturamento: {df_tipo_faturamento.count():,} registros")

print("\n✅ Todas as tabelas carregadas")

## 3. Criação de Views Temporárias

In [ ]:
df_faturamento.createOrReplaceTempView("fato_faturamento")
df_plataforma.createOrReplaceTempView("dim_plataforma")
df_tipo_faturamento.createOrReplaceTempView("dim_tipo_faturamento")

print("✅ Views temporárias criadas")

## 4. SQL Engine - Feature Explosion (~120 Variáveis)

In [ ]:
SQL_TEMPLATE = """
WITH base_enrich AS (
    SELECT 
        f.NUM_CPF,
        CAST('{safra}' AS INT) AS SAFRA,
        f.CONTRATO,
        f.DAT_REFERENCIA,
        f.DAT_CRIACAO_FAT,
        f.DAT_VENCIMENTO_FAT,
        f.DAT_STATUS_FAT,
        f.DAT_ATIVACAO_CONTA_CLI,
        
        -- Valores Financeiros
        COALESCE(f.VAL_FAT_BRUTO, 0) AS VAL_FAT_BRUTO,
        COALESCE(f.VAL_FAT_LIQUIDO, 0) AS VAL_FAT_LIQUIDO,
        COALESCE(f.VAL_FAT_CREDITO, 0) AS VAL_FAT_CREDITO,
        COALESCE(f.VAL_FAT_ABERTO, 0) AS VAL_FAT_ABERTO,
        COALESCE(f.VAL_FAT_ABERTO_LIQ, 0) AS VAL_FAT_ABERTO_LIQ,
        COALESCE(f.VAL_FAT_AJUSTE, 0) AS VAL_FAT_AJUSTE,
        COALESCE(f.VAL_FAT_BRUTO_BC, 0) AS VAL_FAT_BRUTO_BC,
        COALESCE(f.VAL_FAT_PAGAMENTO_BRUTO, 0) AS VAL_FAT_PAGAMENTO_BRUTO,
        COALESCE(f.VAL_FAT_LIQ_JM_MC, 0) AS VAL_FAT_LIQ_JM_MC,
        COALESCE(f.VAL_MULTA_JUROS, 0) AS VAL_MULTA_JUROS,
        COALESCE(f.VAL_MULTA_CANCELAMENTO, 0) AS VAL_MULTA_CANCELAMENTO,
        COALESCE(f.VAL_PARC_APARELHO_LIQ, 0) AS VAL_PARC_APARELHO_LIQ,
        
        -- Indicadores (Flags)
        f.IND_WO,
        f.IND_PDD,
        f.IND_PCCR,
        f.IND_ACA,
        f.IND_PRIMEIRA_FAT,
        f.IND_FRAUDE,
        f.IND_ISENCAO_COB_FAT,
        
        -- Plataforma
        f.COD_PLATAFORMA,
        p.DSC_GRUPO_PLATAFORMA,
        p.COD_GRUPO_PLATAFORMA_BI,
        
        -- Tipo Faturamento
        t.DSC_TIPO_FATURAMENTO
        
    FROM fato_faturamento f
    LEFT JOIN dim_plataforma p ON f.COD_PLATAFORMA = p.COD_PLATAFORMA
    LEFT JOIN dim_tipo_faturamento t ON f.DW_TIPO_FATURAMENTO = t.DW_TIPO_FATURAMENTO
    WHERE f.DAT_REFERENCIA < '{data_cutoff}'
),

agregado AS (
    SELECT 
        NUM_CPF,
        SAFRA,
        
        -- ===== 1. VOLUMETRIA (15 vars) =====
        COUNT(*) AS QTD_FATURAS,
        COUNT(DISTINCT CONTRATO) AS QTD_CONTRATOS_DISTINTOS,
        COUNT(DISTINCT DAT_REFERENCIA) AS QTD_SAFRAS_DISTINTAS,
        SUM(CASE WHEN IND_PRIMEIRA_FAT = 'S' THEN 1 ELSE 0 END) AS QTD_FATURAS_PRIMEIRA,
        SUM(CASE WHEN IND_WO = 'W' THEN 1 ELSE 0 END) AS QTD_FATURAS_WO,
        SUM(CASE WHEN IND_WO = 'R' THEN 1 ELSE 0 END) AS QTD_FATURAS_REGULAR,
        SUM(CASE WHEN IND_PDD = 'S' THEN 1 ELSE 0 END) AS QTD_FATURAS_PDD,
        SUM(CASE WHEN IND_PDD = 'N' THEN 1 ELSE 0 END) AS QTD_FATURAS_SEM_PDD,
        SUM(CASE WHEN IND_PCCR = 'C' THEN 1 ELSE 0 END) AS QTD_FATURAS_PCCR_C,
        SUM(CASE WHEN IND_PCCR = 'W' THEN 1 ELSE 0 END) AS QTD_FATURAS_PCCR_W,
        SUM(CASE WHEN IND_ACA = 'S' THEN 1 ELSE 0 END) AS QTD_FATURAS_ACA,
        SUM(CASE WHEN IND_ACA = 'N' THEN 1 ELSE 0 END) AS QTD_FATURAS_SEM_ACA,
        SUM(CASE WHEN IND_FRAUDE = 'S' THEN 1 ELSE 0 END) AS QTD_FATURAS_FRAUDE,
        SUM(CASE WHEN IND_ISENCAO_COB_FAT IN ('Y','S') THEN 1 ELSE 0 END) AS QTD_FATURAS_ISENTAS,
        SUM(CASE WHEN IND_ISENCAO_COB_FAT = 'N' THEN 1 ELSE 0 END) AS QTD_FATURAS_NAO_ISENTAS,
        
        -- ===== 2. VALORES FINANCEIROS (25 vars) =====
        SUM(VAL_FAT_BRUTO) AS VLR_FAT_BRUTO_TOTAL,
        AVG(VAL_FAT_BRUTO) AS VLR_FAT_BRUTO_MEDIO,
        MAX(VAL_FAT_BRUTO) AS VLR_FAT_BRUTO_MAX,
        MIN(VAL_FAT_BRUTO) AS VLR_FAT_BRUTO_MIN,
        STDDEV(VAL_FAT_BRUTO) AS VLR_FAT_BRUTO_STDDEV,
        
        SUM(VAL_FAT_LIQUIDO) AS VLR_FAT_LIQUIDO_TOTAL,
        AVG(VAL_FAT_LIQUIDO) AS VLR_FAT_LIQUIDO_MEDIO,
        MAX(VAL_FAT_LIQUIDO) AS VLR_FAT_LIQUIDO_MAX,
        
        SUM(VAL_FAT_CREDITO) AS VLR_FAT_CREDITO_TOTAL,
        AVG(VAL_FAT_CREDITO) AS VLR_FAT_CREDITO_MEDIO,
        MAX(VAL_FAT_CREDITO) AS VLR_FAT_CREDITO_MAX,
        
        SUM(VAL_FAT_ABERTO) AS VLR_FAT_ABERTO_TOTAL,
        AVG(VAL_FAT_ABERTO) AS VLR_FAT_ABERTO_MEDIO,
        MAX(VAL_FAT_ABERTO) AS VLR_FAT_ABERTO_MAX,
        
        SUM(VAL_FAT_ABERTO_LIQ) AS VLR_FAT_ABERTO_LIQ_TOTAL,
        AVG(VAL_FAT_ABERTO_LIQ) AS VLR_FAT_ABERTO_LIQ_MEDIO,
        
        SUM(VAL_MULTA_JUROS) AS VLR_MULTA_JUROS_TOTAL,
        AVG(VAL_MULTA_JUROS) AS VLR_MULTA_JUROS_MEDIO,
        MAX(VAL_MULTA_JUROS) AS VLR_MULTA_JUROS_MAX,
        
        SUM(VAL_MULTA_CANCELAMENTO) AS VLR_MULTA_CANCEL_TOTAL,
        AVG(VAL_MULTA_CANCELAMENTO) AS VLR_MULTA_CANCEL_MEDIO,
        
        SUM(VAL_FAT_AJUSTE) AS VLR_FAT_AJUSTE_TOTAL,
        SUM(VAL_FAT_PAGAMENTO_BRUTO) AS VLR_FAT_PAGAMENTO_BRUTO_TOTAL,
        SUM(VAL_PARC_APARELHO_LIQ) AS VLR_PARC_APARELHO_TOTAL,
        SUM(VAL_FAT_LIQ_JM_MC) AS VLR_FAT_LIQ_JM_MC_TOTAL,
        
        -- ===== 3. PIVOT POR PLATAFORMA (20 vars) =====
        SUM(CASE WHEN COD_PLATAFORMA = 'AUTOC' THEN 1 ELSE 0 END) AS QTD_FAT_AUTOC,
        SUM(CASE WHEN COD_PLATAFORMA = 'AUTOC' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_AUTOC,
        SUM(CASE WHEN COD_PLATAFORMA = 'POSPG' THEN 1 ELSE 0 END) AS QTD_FAT_POSPG,
        SUM(CASE WHEN COD_PLATAFORMA = 'POSPG' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_POSPG,
        SUM(CASE WHEN COD_PLATAFORMA = 'PREPG' THEN 1 ELSE 0 END) AS QTD_FAT_PREPG,
        SUM(CASE WHEN COD_PLATAFORMA = 'PREPG' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_PREPG,
        SUM(CASE WHEN COD_PLATAFORMA = 'POSBL' THEN 1 ELSE 0 END) AS QTD_FAT_POSBL,
        SUM(CASE WHEN COD_PLATAFORMA = 'POSBL' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_POSBL,
        SUM(CASE WHEN COD_PLATAFORMA = '-2' THEN 1 ELSE 0 END) AS QTD_FAT_SEM_PLATAFORMA,
        SUM(CASE WHEN COD_PLATAFORMA = '-2' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_SEM_PLATAFORMA,
        
        -- Pivot por Grupo Plataforma
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Pós Pago' THEN 1 ELSE 0 END) AS QTD_FAT_POS_PAGO,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Pós Pago' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_POS_PAGO,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Pré Pago' THEN 1 ELSE 0 END) AS QTD_FAT_PRE_PAGO,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Pré Pago' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_PRE_PAGO,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Controle' THEN 1 ELSE 0 END) AS QTD_FAT_CONTROLE,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Controle' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_CONTROLE,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Telemetria' THEN 1 ELSE 0 END) AS QTD_FAT_TELEMETRIA,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Telemetria' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_TELEMETRIA,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Outros' THEN 1 ELSE 0 END) AS QTD_FAT_OUTROS,
        SUM(CASE WHEN DSC_GRUPO_PLATAFORMA = 'Outros' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_OUTROS,
        
        -- ===== 4. PIVOT POR INDICADORES WO/PDD (12 vars) =====
        SUM(CASE WHEN IND_WO = 'W' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_WO,
        SUM(CASE WHEN IND_WO = 'R' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_REGULAR,
        SUM(CASE WHEN IND_PDD = 'S' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_PDD,
        SUM(CASE WHEN IND_PDD = 'N' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_SEM_PDD,
        SUM(CASE WHEN IND_ACA = 'S' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_ACA,
        SUM(CASE WHEN IND_FRAUDE = 'S' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_FAT_FRAUDE,
        SUM(CASE WHEN IND_PRIMEIRA_FAT = 'S' THEN VAL_FAT_LIQUIDO ELSE 0 END) AS VLR_PRIMEIRA_FAT,
        AVG(CASE WHEN IND_WO = 'W' THEN VAL_FAT_LIQUIDO END) AS VLR_MEDIO_WO,
        AVG(CASE WHEN IND_PDD = 'S' THEN VAL_FAT_LIQUIDO END) AS VLR_MEDIO_PDD,
        AVG(CASE WHEN IND_PRIMEIRA_FAT = 'S' THEN VAL_FAT_LIQUIDO END) AS VLR_MEDIO_PRIMEIRA_FAT,
        MAX(CASE WHEN IND_WO = 'W' THEN VAL_FAT_LIQUIDO END) AS VLR_MAX_WO,
        MAX(CASE WHEN IND_PDD = 'S' THEN VAL_FAT_LIQUIDO END) AS VLR_MAX_PDD,
        
        -- ===== 5. MÉTRICAS TEMPORAIS (15 vars) =====
        DATEDIFF(TO_DATE('{data_cutoff}'), MIN(DAT_CRIACAO_FAT)) AS DIAS_DESDE_PRIMEIRA_FAT,
        DATEDIFF(TO_DATE('{data_cutoff}'), MAX(DAT_CRIACAO_FAT)) AS DIAS_DESDE_ULTIMA_FAT,
        DATEDIFF(MAX(DAT_CRIACAO_FAT), MIN(DAT_CRIACAO_FAT)) AS DIAS_ENTRE_PRIMEIRA_ULTIMA_FAT,
        DATEDIFF(TO_DATE('{data_cutoff}'), MIN(DAT_ATIVACAO_CONTA_CLI)) AS DIAS_DESDE_ATIVACAO_CONTA,
        AVG(DATEDIFF(DAT_VENCIMENTO_FAT, DAT_CRIACAO_FAT)) AS DIAS_MEDIO_CRIACAO_VENCIMENTO,
        MAX(DATEDIFF(DAT_VENCIMENTO_FAT, DAT_CRIACAO_FAT)) AS DIAS_MAX_CRIACAO_VENCIMENTO,
        MIN(DATEDIFF(DAT_VENCIMENTO_FAT, DAT_CRIACAO_FAT)) AS DIAS_MIN_CRIACAO_VENCIMENTO,
        AVG(DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT)) AS DIAS_ATRASO_MEDIO,
        MAX(DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT)) AS DIAS_ATRASO_MAX,
        MIN(DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT)) AS DIAS_ATRASO_MIN,
        SUM(CASE WHEN DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT) > 0 THEN 1 ELSE 0 END) AS QTD_FATURAS_ATRASADAS,
        SUM(CASE WHEN DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT) > 30 THEN 1 ELSE 0 END) AS QTD_FATURAS_ATRASO_30D,
        SUM(CASE WHEN DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT) > 60 THEN 1 ELSE 0 END) AS QTD_FATURAS_ATRASO_60D,
        SUM(CASE WHEN DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT) > 90 THEN 1 ELSE 0 END) AS QTD_FATURAS_ATRASO_90D,
        SUM(CASE WHEN DATEDIFF(DAT_STATUS_FAT, DAT_VENCIMENTO_FAT) <= 0 THEN 1 ELSE 0 END) AS QTD_FATURAS_EM_DIA
        
    FROM base_enrich
    GROUP BY NUM_CPF, SAFRA
)

SELECT 
    a.*,
    
    -- ===== 7. TAXAS E RATIOS (15 vars) =====
    ROUND(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_WO,
    ROUND(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_PDD,
    ROUND(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ACA,
    ROUND(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_FRAUDE,
    ROUND(a.QTD_FATURAS_PRIMEIRA / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_PRIMEIRA_FAT,
    ROUND(a.QTD_FATURAS_ISENTAS / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ISENCAO,
    ROUND(a.QTD_FATURAS_ATRASADAS / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ATRASO,
    ROUND(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ATRASO_30D,
    ROUND(a.QTD_FATURAS_ATRASO_60D / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ATRASO_60D,
    ROUND(a.QTD_FATURAS_ATRASO_90D / NULLIF(a.QTD_FATURAS, 0), 4) AS TAXA_ATRASO_90D,
    ROUND(a.VLR_FAT_CREDITO_TOTAL / NULLIF(a.VLR_FAT_BRUTO_TOTAL, 0), 4) AS TAXA_CREDITO_BRUTO,
    ROUND(a.VLR_FAT_ABERTO_TOTAL / NULLIF(a.VLR_FAT_BRUTO_TOTAL, 0), 4) AS TAXA_ABERTO_BRUTO,
    ROUND(a.VLR_MULTA_JUROS_TOTAL / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS TAXA_MULTA_JUROS,
    ROUND(a.VLR_FAT_WO / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS TAXA_VLR_WO,
    ROUND(a.VLR_FAT_PDD / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS TAXA_VLR_PDD,
    
    -- ===== 8. ÍNDICES DE ESTABILIDADE (8 vars) =====
    ROUND(a.VLR_FAT_BRUTO_STDDEV / NULLIF(a.VLR_FAT_BRUTO_MEDIO, 0), 4) AS COEF_VAR_FAT_BRUTO,
    ROUND(a.VLR_FAT_BRUTO_MAX / NULLIF(a.VLR_FAT_BRUTO_TOTAL, 0), 4) AS INDICE_CONCENTRACAO_FAT,
    ROUND(GREATEST(a.VLR_FAT_AUTOC, a.VLR_FAT_POSPG, a.VLR_FAT_PREPG, a.VLR_FAT_POSBL) / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS INDICE_CONCENTRACAO_PLATAFORMA,
    ROUND((a.VLR_FAT_BRUTO_MAX - a.VLR_FAT_BRUTO_MIN) / NULLIF(a.VLR_FAT_BRUTO_MEDIO, 0), 4) AS AMPLITUDE_RELATIVA_FAT,
    ROUND(a.VLR_FAT_POS_PAGO / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS SHARE_WALLET_POS_PAGO,
    ROUND(a.VLR_FAT_PRE_PAGO / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS SHARE_WALLET_PRE_PAGO,
    ROUND(a.VLR_FAT_CONTROLE / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS SHARE_WALLET_CONTROLE,
    ROUND(a.VLR_FAT_TELEMETRIA / NULLIF(a.VLR_FAT_LIQUIDO_TOTAL, 0), 4) AS SHARE_WALLET_TELEMETRIA,
    
    -- ===== 9. SCORE DE RISCO (Composto 0-100) =====
    ROUND(
        LEAST(100, GREATEST(0,
            -- Peso: WO (30%) + PDD (25%) + Atraso (20%) + Fraude (15%) + ACA (10%)
            (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30) +
            (COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25) +
            (COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20) +
            (COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15) +
            (COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10)
        ) * 100)
    , 2) AS SCORE_RISCO,
    
    -- ===== 10. FLAGS E SEGMENTAÇÃO =====
    CASE 
        WHEN (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30 +
              COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25 +
              COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20 +
              COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15 +
              COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10) * 100 >= 70 THEN 1 
        ELSE 0 
    END AS FLAG_ALTO_RISCO,
    
    CASE 
        WHEN (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30 +
              COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25 +
              COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20 +
              COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15 +
              COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10) * 100 < 30 THEN 1 
        ELSE 0 
    END AS FLAG_BAIXO_RISCO,
    
    CASE 
        WHEN (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30 +
              COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25 +
              COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20 +
              COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15 +
              COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10) * 100 >= 80 THEN 'CRITICO'
        WHEN (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30 +
              COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25 +
              COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20 +
              COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15 +
              COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10) * 100 >= 50 THEN 'ALTO'
        WHEN (COALESCE(a.QTD_FATURAS_WO / NULLIF(a.QTD_FATURAS, 0), 0) * 30 +
              COALESCE(a.QTD_FATURAS_PDD / NULLIF(a.QTD_FATURAS, 0), 0) * 25 +
              COALESCE(a.QTD_FATURAS_ATRASO_30D / NULLIF(a.QTD_FATURAS, 0), 0) * 20 +
              COALESCE(a.QTD_FATURAS_FRAUDE / NULLIF(a.QTD_FATURAS, 0), 0) * 15 +
              COALESCE(a.QTD_FATURAS_ACA / NULLIF(a.QTD_FATURAS, 0), 0) * 10) * 100 >= 30 THEN 'MEDIO'
        ELSE 'BAIXO'
    END AS SEGMENTO_RISCO,
    
    current_timestamp() AS DT_PROCESSAMENTO
    
FROM agregado a
"""

print("✅ Query SQL definida (sem features cadastrais - leakage fix)")

## 5. Execução e Cache

In [ ]:
# =============================================================================
# LOOP PRINCIPAL - PROCESSAR CADA SAFRA
# =============================================================================
resultados = []

for i, safra in enumerate(LISTA_SAFRAS):
    print(f'\n[{i+1}/{len(LISTA_SAFRAS)}] Processando SAFRA {safra}...')
    data_cutoff = safra_to_cutoff(safra)
    print(f'  Cutoff: {data_cutoff}')
    
    # Gerar query com parametros
    query = SQL_TEMPLATE.format(safra=safra, data_cutoff=data_cutoff)
    
    # Executar
    df_safra = spark.sql(query)
    df_safra.cache()
    cnt = df_safra.count()
    print(f'  Registros: {cnt:,}')
    resultados.append({'safra': safra, 'cnt': cnt})
    
    # Salvar
    modo = 'overwrite' if i == 0 and MODO_ESCRITA == 'overwrite' else 'append'
    df_safra.write.format('delta').mode(modo).partitionBy('SAFRA').option('overwriteSchema', 'true' if modo == 'overwrite' else 'false').save(PATH_OUTPUT)
    print(f'  Salvo (modo: {modo})')
    df_safra.unpersist()

print('\n' + '='*50)
print('CONCLUIDO!')
total = sum(r['cnt'] for r in resultados)
print(f'Total: {total:,} registros em {len(LISTA_SAFRAS)} safras')

## Dicionário de Variáveis

> **Audit Note (2026-02-08)**: Seção 6 "Cadastrais" removida — continha `VLR_FPD = MAX(FPD)` que era data leakage (cópia direta do target). Features cadastrais (STATUSRF, PROD, var_03/04/05, flag_mig2) já existem na tabela base `dados_cadastrais` e não devem ser duplicadas nos books. Ver Story 1.1.

### Volumetria (15 vars)
| Variável | Descrição |
|----------|----------|
| `QTD_FATURAS` | Total de faturas do cliente na safra |
| `QTD_CONTRATOS_DISTINTOS` | Quantidade de contratos distintos |
| `QTD_FATURAS_WO` | Faturas com Write-Off |
| `QTD_FATURAS_PDD` | Faturas com Provisão (PDD) |
| `QTD_FATURAS_FRAUDE` | Faturas marcadas como fraude |
| `QTD_FATURAS_ACA` | Faturas em ação de cobrança |
| `QTD_FATURAS_ATRASADAS` | Faturas pagas após vencimento |
| ... | ... |

### Valores Financeiros (25 vars)
| Variável | Descrição |
|----------|----------|
| `VLR_FAT_BRUTO_TOTAL` | Soma do valor bruto faturado |
| `VLR_FAT_LIQUIDO_TOTAL` | Soma do valor líquido faturado |
| `VLR_FAT_ABERTO_TOTAL` | Soma do valor em aberto |
| `VLR_MULTA_JUROS_TOTAL` | Soma de multas e juros |
| ... | ... |

### Pivot por Plataforma (20 vars)
| Variável | Descrição |
|----------|----------|
| `QTD_FAT_AUTOC` | Faturas da plataforma AUTOC |
| `VLR_FAT_POSPG` | Valor faturado em Pós-Pago |
| `SHARE_WALLET_POS_PAGO` | % do wallet em Pós-Pago |
| ... | ... |

### Indicadores de Risco (15 vars)
| Variável | Descrição |
|----------|----------|
| `TAXA_WO` | % de faturas com Write-Off |
| `TAXA_PDD` | % de faturas com PDD |
| `TAXA_ATRASO_30D` | % de faturas com atraso > 30 dias |
| ... | ... |

### Métricas Temporais (15 vars)
| Variável | Descrição |
|----------|----------|
| `DIAS_DESDE_PRIMEIRA_FAT` | Dias desde primeira fatura |
| `DIAS_ATRASO_MEDIO` | Média de dias de atraso |
| `DIAS_DESDE_ATIVACAO_CONTA` | Tempo de relacionamento |
| ... | ... |

### Índices de Estabilidade (8 vars)
| Variável | Descrição |
|----------|----------|
| `COEF_VAR_FAT_BRUTO` | Coeficiente de variação (StdDev/Mean) |
| `INDICE_CONCENTRACAO_PLATAFORMA` | Concentração em uma plataforma |
| ... | ... |

### Score de Risco (5 vars)
| Variável | Descrição |
|----------|----------|
| `SCORE_RISCO` | Score de risco (0-100) — baseado em WO, PDD, atraso, fraude, ACA |
| `FLAG_ALTO_RISCO` | 1 se Score >= 70 |
| `FLAG_BAIXO_RISCO` | 1 se Score < 30 |
| `SEGMENTO_RISCO` | CRITICO/ALTO/MEDIO/BAIXO |
| `DT_PROCESSAMENTO` | Data/hora de geração |

### Variáveis REMOVIDAS (leakage fix — Story 1.1)
| Variável | Motivo da Remoção |
|----------|-------------------|
| ~~`VLR_FPD`~~ | **DATA LEAKAGE** — era MAX(FPD), cópia direta do target |
| ~~`FLAG_STATUSRF_*`~~ | Duplicata de dados_cadastrais |
| ~~`FLAG_PROD_*`~~ | Duplicata de dados_cadastrais |
| ~~`VLR_VAR_03/04/05`~~ | Duplicata de dados_cadastrais |
| ~~`FLAG_MIG2_AQUISICAO`~~ | Duplicata de dados_cadastrais |